# 3주차 — 병역학 규칙 기반 경보


## 학습 목표
- 임계치 규칙 설계와 성능 기초 평가

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from utils import risk_index_rulebased

weather = pd.read_csv(str(Path("..")/"data"/"sample_weather_2022.csv"))
weather['date'] = pd.to_datetime(weather['date'])

# 규칙 기반 경보(토이)
risk = weather.apply(risk_index_rulebased, axis=1)
weather['risk_index'] = risk

# 이동합으로 연속 위험일수 개념
weather['risk_7d_sum'] = weather['risk_index'].rolling(7, min_periods=1).sum()

plt.figure(figsize=(10,4))
plt.plot(weather['date'], weather['risk_7d_sum'])
plt.title("7일 위험일수 합")
plt.xlabel("날짜"); plt.ylabel("days")
plt.tight_layout(); plt.show()
